In [61]:
import pandas as pd
from os import path
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.utils.fixes import loguniform    
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
import pickle
import numpy as np
import os
import json
import csv

In [2]:
DATA_DIR = "../../data"

In [22]:
chen_train = pd.read_csv(path.join(DATA_DIR, "chen/deduplicated/chen_train_data.csv"), index_col=0)
chen_valid = pd.read_csv(path.join(DATA_DIR, "chen/deduplicated/chen_valid_data.csv"), index_col=0)
#chen_train = pd.concat([chen_train, chen_valid])
chen_train.head()
# should there be a concaenation???

,Antibody_ID,heavy,light,Y
2073,6aod,EVQLVQSGAEVKKPGASVKVSCKASGYTFTGYYMHWVRQAPGQGLE...,DIVMTKSPSSLSASVGDRVTITCRASQGIRNDLGWYQQKPGKAPKR...,0
1517,4yny,EVQLVESGGGLVQPGRSLKLSCAASGFTFSNYGMAWVRQTPTKGLE...,EFVLTQPNSVSTNLGSTVKLSCKRSTGNIGSNYVNWYQQHEGRSPT...,1
2025,5xcv,EVQLVESGGGLVQPGRSLKLSCAASGFTFSNYGMAWVRQTPTKGLE...,QFVLTQPNSVSTNLGSTVKLSCKRSTGNIGSNYVNWYQQHEGRSPT...,1
2070,6and,EVQLVESGGGLVQPGGSLRLSCAASGYEFSRSWMNWVRQAPGKGLE...,DIQMTQSPSSLSASVGDRVTITCRSSQSIVHSVGNTFLEWYQQKPG...,1
666,2xqy,QVQLQQPGAELVKPGASVKMSCKASGYSFTSYWMNWVKQRPGRGLE...,DIVLTQSPASLALSLGQRATISCRASKSVSTSGYSYMYWYQQKPGQ...,0


In [4]:
chen_test = pd.read_csv(path.join(DATA_DIR, "chen/deduplicated/chen_test_data.csv"))

In [10]:
tap_data = pd.read_csv(path.join(DATA_DIR, "tap/TAP_data.csv"))
tap_data.head()

,Antibody_ID,heavy,light,CDR_length,PSH,PPC,PNC,SFvCSP,Y
0,Abagovomab,QVKLQESGAELARPGASVKLSCKASGYTFTNYWMQWVKQRPGQGLD...,DIELTQSPASLSASVGETVTITCQASENIYSYLAWHQQKQGKSPQL...,46,129.7603,0.0000,0.0000,16.32,1
1,Abituzumab,QVQLQQSGGELAKPGASVKVSCKASGYTFSSFWMHWVRQAPGQGLE...,DIQMTQSPSSLSASVGDRVTITCRASQDISNYLAWYQQKPGKAPKL...,45,115.9106,0.0954,0.0421,-3.10,1
2,Abrilumab,QVQLVQSGAEVKKPGASVKVSCKVSGYTLSDLSIHWVRQAPGKGLE...,DIQMTQSPSSVSASVGDRVTITCRASQGISSWLAWYQQKPGKAPKL...,45,109.6995,0.0000,0.8965,-4.00,1
3,Actoxumab,QVQLVESGGGVVQPGRSLRLSCAASGFSFSNYGMHWVRQAPGKGLE...,DIQMTQSPSSVSASVGDRVTITCRASQGISSWLAWYQHKPGKAPKL...,49,112.6290,0.0000,1.1247,3.10,1
4,Adalimumab,EVQLVESGGGLVQPGRSLRLSCAASGFTFDDYAMHWVRQAPGKGLE...,DIQMTQSPSSLSASVGDRVTITCRASQGIRNYLAWYQQKPGKAPKL...,48,111.2512,0.0485,1.1364,-19.50,1


In [18]:
chen_similarity = pd.read_csv(path.join(DATA_DIR, "chen/distances/deduplicated_anarci_similarity.csv"), index_col=0)
chen_similarity.head()

,0,1,2,3,4,5,7,8,9,10,...,2398,2400,2401,2402,2403,2404,2405,2406,2407,2408
0,1.00000,0.77700,0.70350,0.69995,0.65085,0.67350,0.66915,0.66915,0.76100,0.71800,...,0.59250,0.63185,0.65150,0.61850,0.70070,0.60550,0.54015,0.70460,0.61020,0.62840
1,0.77700,1.00000,0.75000,0.73500,0.69315,0.70240,0.65570,0.65570,0.74785,0.79775,...,0.59600,0.65265,0.67200,0.63085,0.69350,0.60750,0.55310,0.71665,0.63920,0.63185
2,0.70350,0.75000,1.00000,0.76100,0.69315,0.68970,0.64550,0.64550,0.70850,0.72070,...,0.55975,0.60500,0.65050,0.62120,0.69350,0.58815,0.55555,0.69965,0.57580,0.60875
3,0.69995,0.73500,0.76100,1.00000,0.66160,0.68700,0.65500,0.65500,0.68315,0.71335,...,0.57800,0.63665,0.65900,0.61900,0.70700,0.59300,0.54950,0.67185,0.58200,0.61370
4,0.65085,0.69315,0.69315,0.66160,1.00000,0.63025,0.64000,0.64000,0.64965,0.68375,...,0.62485,0.64180,0.60255,0.60010,0.67985,0.61990,0.54340,0.76250,0.62135,0.63570


# Models

In [20]:
# different usage from the other models
def knn(n):
    model = NearestNeighbors(metric="precomputed")
    parameters = {'n_neighbors': [1,3,5]}
    return model, parameters, "kNN"

In [15]:
def logistic_regression(n):
    lr = LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42)
    parameters = {'C':loguniform(0.001, 1000), 'penalty': ["l2"], "solver": ["lbfgs", "sag"]}
    return lr, parameters, "logistic_regression"

In [16]:
def random_forest(n):
    rf = RandomForestClassifier(random_state=42, n_jobs=-1, class_weight='balanced')
    parameters = {'n_estimators': np.arange(1, 200, 10), 'max_depth': np.arange(1, min(50,n), 2), 
                  'max_features': np.arange(0.1, 0.75, 0.05)}
    return rf, parameters, "random_forest"

In [21]:
def multilayer_perceptron(n):
    mlp = MLPClassifier(random_state=42, max_iter=int(1000))
    parameters = {'hidden_layer_sizes': [(100,), (50,), (100, 100)], "activation": ["relu", "logistic"]}
    return mlp, parameters, "multilayer_perceptron"

In [23]:
def svm(n):
    svc = SVC(max_iter=8000, probability=True, class_weight='balanced')
    parameters = {'C': loguniform(0.001, 100), 'kernel':["linear", "rbf"], 'gamma': loguniform(1e-3, 1e0)}
    return svc, parameters, "SVM"

In [24]:
def gradient_boosting(n):
    gb = GradientBoostingClassifier(random_state=42, n_iter_no_change=70)
    parameters = {'learning_rate': loguniform(0.01, 0.5), 
                  'n_estimators': np.arange(1, 200, 10), 
                  'max_depth': np.arange(1, min(20,n), 2), 
                  'max_features': np.arange(0.1, 0.6, 0.1)}
    return gb, parameters, "gradient_boosting"

In [ ]:
def output_evaluation(model_type, params, metrics, data, outpath, preprocessing=None):
    prepro = "_"+preprocessing if preprocessing is not None else ""
    filename = os.path.join(DATA_DIR, "evaluations", outpath, f"{model_type}_{data}{prepro}.json")
    out_dict = {
        "model_type": model_type,
        "data": data
    }
    out_dict["params"] = {}
    for key, value in params.items():
        out_dict["params"]["key"] = str(value)
    out_dict["metrics"] = metrics
    out_dict["preprocessing"] = "none" if preprocessing is None else preprocessing
    
    json.dump(out_dict, open(filename, "w"))
    
    filename_sum = os.path.join(DATA_DIR, f"evaluations/{outpath}/all.csv")
    #"../evaluations/all.csv"
    line = [model_type, data, out_dict["preprocessing"], metrics["f1"], metrics["mcc"], metrics["acc"],metrics["precision"],metrics["recall"],metrics["auc"], filename]
    with open(filename_sum, 'a', newline='') as csvfile:
        csvwriter = csv.writer(csvfile, delimiter='\t')
        csvwriter.writerow(line)

In [55]:
def train_and_eval(model_name, classifier, parameters, X_train, y_train, X_valid, y_valid, 
                   data_name, outpath, preprocessing=None):
    grid = RandomizedSearchCV(classifier, parameters, verbose=1, scoring="f1")
    grid.fit(X_train, y_train)
    estimator = grid.best_estimator_
    best_params = grid.best_params_
    y_pred = estimator.predict(X_valid)
    prepro = "_"+preprocessing if preprocessing is not None else ""
    filename = path.join(DATA_DIR, "evaluations", outpath, "models", f"{model_name}_{data_name}{prepro}.pkl")
    with open(filename, 'wb') as f:
        pickle.dump(estimator, f)
    #pickle.dump(estimator, open(filename, "w"))
    metric_dict = {
        "f1": float(metrics.f1_score(y_valid, y_pred)),
        "acc": float(metrics.accuracy_score(y_valid, y_pred)),
        "mcc": float(metrics.matthews_corrcoef(y_valid, y_pred)),
        "auc": float(metrics.roc_auc_score(y_valid, y_pred)),
        "precision": float(metrics.precision_score(y_valid, y_pred)),
        "recall": float(metrics.recall_score(y_valid, y_pred))
    }
    
    print(f"{model_name}, {data_name}")
    print(f"F1: {metric_dict['f1']}")
    print(f"MCC: {metric_dict['mcc']}")
    print(f"Accuracy: {metric_dict['acc']}")
    print(f"Precision: {metric_dict['precision']}")
    print(f"Recall: {metric_dict['recall']}")
    print(f"AUC: {metric_dict['auc']}")
    print(f"-----")
    
    output_evaluation(model_name, parameters, metric_dict, data_name, outpath, preprocessing=None)

In [82]:
def try_all(X_train, y_train, X_valid, y_valid, data_name, outpath, preprocessing=None):
    n = len(y_train)
    for model_creator in [logistic_regression, random_forest, gradient_boosting, svm, multilayer_perceptron]:
        classifier, params, model_label = model_creator(n)
        print("\n")
        print(f'Training model {model_label} on data {data_name} \n')
        train_and_eval(model_label, classifier, params, X_train, y_train, X_valid, y_valid, 
                   data_name, outpath, preprocessing=None)

# Numbering

In [66]:
x_chen = pd.read_csv(path.join(DATA_DIR, "chen/integer_encoding/chen_integer_encoded.csv"), index_col=0)
x_chen_train = x_chen.loc[chen_train.index]
x_chen_test = x_chen.loc[chen_test.index]
x_chen_valid = x_chen.loc[chen_valid.index]
x_chen_train.head()

,Ab_ID,0,1,2,3,4,5,6,7,8,...,271,272,273,274,275,276,277,278,279,280
2073,6aod,4,18,14,10,18,14,16,6,1,...,0,0,0,0,0,0,0,0,0,0
1517,4yny,4,18,14,10,18,4,16,6,6,...,0,0,0,0,0,0,0,0,0,0
2025,5xcv,4,18,14,10,18,4,16,6,6,...,0,0,0,0,0,0,0,0,0,0
2070,6and,4,18,14,10,18,4,16,6,6,...,0,0,0,0,0,0,0,0,0,0
666,2xqy,14,18,14,10,14,14,13,6,1,...,0,0,0,0,0,0,0,0,0,0


In [67]:
len(x_chen_train)

1338

In [68]:
x_tap = pd.read_csv(path.join(DATA_DIR, "tap/integer_encoding/tap_integer_encoded.csv"))
x_tap.head()

,Ab_ID,0,1,2,3,4,5,6,7,8,...,271,272,273,274,275,276,277,278,279,280
0,Abagovomab,14,18,9,10,14,4,16,6,1,...,0,0,0,0,0,0,0,0,0,0
1,Abituzumab,14,18,14,10,14,14,16,6,6,...,0,0,0,0,0,0,0,0,0,0
2,Abrilumab,14,18,14,10,18,14,16,6,1,...,0,0,0,0,0,0,0,0,0,0
3,Actoxumab,14,18,14,10,18,4,16,6,6,...,0,0,0,0,0,0,0,0,0,0
4,Adalimumab,4,18,14,10,18,4,16,6,6,...,0,0,0,0,0,0,0,0,0,0


In [69]:
try_all(x_chen_train.drop("Ab_ID", axis=1), chen_train["Y"], x_chen_valid.drop("Ab_ID", axis=1), chen_valid["Y"], "integer_encoded", "2021-11-24", preprocessing=None)



Training model logistic_regression on data integer_encoded 

Fitting 5 folds for each of 10 candidates, totalling 50 fits


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/

logistic_regression, integer_encoded
F1: 0.5074626865671641
MCC: 0.34618072417297263
Accuracy: 0.725
Precision: 0.4146341463414634
Recall: 0.6538461538461539
AUC: 0.6992635024549918
-----


Training model random_forest on data integer_encoded 

Fitting 5 folds for each of 10 candidates, totalling 50 fits
random_forest, integer_encoded
F1: 0.2285714285714286
MCC: 0.1574351717262429
Accuracy: 0.775
Precision: 0.4444444444444444
Recall: 0.15384615384615385
AUC: 0.5503273322422257
-----


Training model gradient_boosting on data integer_encoded 

Fitting 5 folds for each of 10 candidates, totalling 50 fits
gradient_boosting, integer_encoded
F1: 0.30303030303030304
MCC: 0.3006335458732897
Accuracy: 0.8083333333333333
Precision: 0.7142857142857143
Recall: 0.19230769230769232
AUC: 0.5855155482815056
-----


Training model SVM on data integer_encoded 

Fitting 5 folds for each of 10 candidates, totalling 50 fits


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=8000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=8000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=8000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=8000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_it

SVM, integer_encoded
F1: 0.4406779661016949
MCC: 0.26501466996213197
Accuracy: 0.725
Precision: 0.3939393939393939
Recall: 0.5
AUC: 0.6436170212765958
-----


Training model multilayer_perceptron on data integer_encoded 

Fitting 5 folds for each of 6 candidates, totalling 30 fits
multilayer_perceptron, integer_encoded
F1: 0.4827586206896552
MCC: 0.32324284925164637
Accuracy: 0.75
Precision: 0.4375
Recall: 0.5384615384615384
AUC: 0.6734860883797054
-----


# PyBioMed

In [78]:
x_chen = pd.read_feather(path.join(DATA_DIR, "chen/pybiomed/X_data.ftr"))
x_chen_train = x_chen.loc[chen_train.index]
x_chen_test = x_chen.loc[chen_test.index]
x_chen_valid = x_chen.loc[chen_valid.index]
x_chen_train.head()

,Ab_ID,0,1,2,3,4,5,6,7,8,...,19750,19751,19752,19753,19754,19755,19756,19757,19758,19759
2073,6aod,7.692,5.983,0.855,5.983,1.709,4.274,5.983,10.256,0.855,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1517,4yny,6.838,5.128,1.709,5.128,1.709,3.419,4.274,11.111,0.855,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025,5xcv,6.838,5.128,1.709,5.128,2.564,3.419,4.274,11.111,0.855,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2070,6and,6.667,5.833,2.500,5.833,1.667,4.167,5.000,13.333,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
666,2xqy,3.390,3.390,2.542,5.932,1.695,3.390,6.780,9.322,0.847,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [79]:
x_tap = pd.read_feather(path.join(DATA_DIR, "tap/pybiomed/X_TAP_data.ftr"))
x_tap.head()

,0,1,2,3,4,5,6,7,8,9,...,19750,19751,19752,19753,19754,19755,19756,19757,19758,19759
0,10.084,3.361,2.521,3.361,1.681,3.361,5.882,11.765,0.840,1.681,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7.627,4.237,1.695,3.390,1.695,5.085,5.932,10.169,0.847,2.542,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5.932,1.695,0.000,5.932,1.695,5.085,6.780,10.169,0.847,1.695,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5.738,6.557,4.098,5.738,1.639,3.279,4.918,11.475,0.820,3.279,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,9.091,4.959,3.306,5.785,1.653,4.132,4.132,9.091,1.653,2.479,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [81]:
try_all(x_chen_train.drop("Ab_ID", axis=1), chen_train["Y"], x_chen_valid.drop("Ab_ID", axis=1), chen_valid["Y"], "pybiomed", "2021-11-24", preprocessing=None)



Training model random_forest on data pybiomed 

Fitting 5 folds for each of 10 candidates, totalling 50 fits


KeyboardInterrupt: 

# Protparam

In [83]:
x_chen = pd.read_csv(path.join(DATA_DIR, "chen/protparam/protparam_features.csv"))
x_chen.rename({"Unnamed: 0": "Ab_ID"}, axis=1, inplace=True)
x_chen_train = x_chen.loc[chen_train.index]
x_chen_test = x_chen.loc[chen_test.index]
x_chen_valid = x_chen.loc[chen_valid.index]
x_chen_train.head()

,Ab_ID,aa_percent0_x,aa_percent1_x,aa_percent2_x,aa_percent3_x,aa_percent4_x,aa_percent5_x,aa_percent6_x,aa_percent7_x,aa_percent8_x,...,flexibility_y,isoelectric_y,mol_extinct1_y,mol_extinct2_y,mw_y,gravy_y,ss_faction1_y,ss_faction2_y,ss_faction3_y,name
2073,1xiw,0.049180,0.016393,0.049180,0.040984,0.032787,0.090164,0.008197,0.016393,0.081967,...,1.000398,6.756409,19940,20065,11853.0516,-0.441121,0.289720,0.280374,0.168224,1xiw
1517,5b3j,0.057377,0.016393,0.032787,0.040984,0.024590,0.114754,0.000000,0.024590,0.040984,...,1.001878,6.753845,17420,17545,11736.0293,-0.425472,0.301887,0.301887,0.160377,5b3j
2025,1j1x,0.026549,0.017699,0.070796,0.026549,0.008850,0.070796,0.000000,0.035398,0.035398,...,1.002679,6.911682,16960,17085,11598.7277,-0.327103,0.280374,0.364486,0.158879,1j1x
2070,5f7e,0.067227,0.016807,0.050420,0.033613,0.033613,0.100840,0.008403,0.016807,0.042017,...,1.004044,4.570740,14440,14565,11255.3375,-0.405825,0.281553,0.300971,0.145631,5f7e
666,5b71,0.097561,0.016260,0.040650,0.032520,0.016260,0.113821,0.024390,0.016260,0.040650,...,1.006930,7.952576,12950,13075,11559.6424,-0.365455,0.245455,0.354545,0.154545,5b71


In [85]:
try_all(x_chen_train.drop(["Ab_ID", "name"], axis=1), chen_train["Y"], x_chen_valid.drop(["Ab_ID", "name"], axis=1), chen_valid["Y"], "protparam", "2021-11-24", preprocessing=None)



Training model logistic_regression on data protparam 

Fitting 5 folds for each of 10 candidates, totalling 50 fits


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:3

logistic_regression, protparam
F1: 0.28235294117647064
MCC: -0.031694676944994615
Accuracy: 0.49166666666666664
Precision: 0.2033898305084746
Recall: 0.46153846153846156
AUC: 0.48076923076923084
-----


Training model random_forest on data protparam 

Fitting 5 folds for each of 10 candidates, totalling 50 fits


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


random_forest, protparam
F1: 0.2285714285714286
MCC: -0.06436267532181093
Accuracy: 0.55
Precision: 0.18181818181818182
Recall: 0.3076923076923077
AUC: 0.4623567921440262
-----


Training model gradient_boosting on data protparam 

Fitting 5 folds for each of 10 candidates, totalling 50 fits
gradient_boosting, protparam
F1: 0.0625
MCC: -0.027843501462510502
Accuracy: 0.75
Precision: 0.16666666666666666
Recall: 0.038461538461538464
AUC: 0.4926350245499182
-----


Training model SVM on data protparam 

Fitting 5 folds for each of 10 candidates, totalling 50 fits


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=8000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=8000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=8000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=8000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_it

SVM, protparam
F1: 0.29126213592233013
MCC: -0.07101027234304307
Accuracy: 0.39166666666666666
Precision: 0.19480519480519481
Recall: 0.5769230769230769
AUC: 0.45867430441898527
-----


Training model multilayer_perceptron on data protparam 

Fitting 5 folds for each of 6 candidates, totalling 30 fits
multilayer_perceptron, protparam
F1: 0.2564102564102564
MCC: 0.1420977252150067
Accuracy: 0.7583333333333333
Precision: 0.38461538461538464
Recall: 0.19230769230769232
AUC: 0.5536006546644844
-----


# BERT Embeddings

In [86]:
x_chen = pd.read_feather(path.join(DATA_DIR, "chen/embeddings/bert/bert_chen_embeddings.ftr"))
x_chen_train = x_chen.loc[chen_train.index]
x_chen_test = x_chen.loc[chen_test.index]
x_chen_valid = x_chen.loc[chen_valid.index]
x_chen_train.head()

,Ab_ID,0,1,2,3,4,5,6,7,8,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
2073,6aod,0.020196,-0.020793,-0.010549,-0.038375,0.066936,-0.022921,0.017874,-0.088493,-0.018926,...,0.001973,-0.003466,-0.014579,0.007682,-0.027888,0.026192,-0.040531,-0.010679,0.007133,-0.027332
1517,4yny,-0.007447,-0.058638,0.013139,-0.029136,0.074901,-0.025573,0.035983,-0.105274,-0.014490,...,0.008387,0.007312,-0.013750,-0.003369,0.007132,0.042410,-0.044542,-0.034221,-0.023036,-0.013044
2025,5xcv,-0.005372,-0.056795,0.012577,-0.031566,0.071131,-0.023377,0.037339,-0.102565,-0.009340,...,0.008306,0.003619,-0.015327,-0.003542,0.003720,0.043285,-0.046977,-0.033989,-0.025775,-0.012205
2070,6and,-0.004229,-0.043975,0.003673,-0.031739,0.070902,-0.022174,0.038274,-0.111665,-0.030382,...,0.006203,-0.016125,-0.021334,-0.005837,-0.025217,0.024086,-0.032259,-0.023480,-0.013043,-0.035664
666,2xqy,0.003754,-0.018278,-0.001506,-0.037899,0.050809,-0.026173,0.003026,-0.090757,0.005515,...,-0.003720,0.004341,-0.036547,-0.009119,-0.038372,0.023391,-0.030063,-0.003554,0.000142,-0.041985


In [ ]:
try_all(x_chen_train.drop("Ab_ID", axis=1), chen_train["Y"], x_chen_valid.drop("Ab_ID", axis=1), chen_valid["Y"], "bert", "2021-11-24", preprocessing=None)



Training model logistic_regression on data bert 

Fitting 5 folds for each of 10 candidates, totalling 50 fits


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:3

logistic_regression, bert
F1: 0.53125
MCC: 0.3812118695553192
Accuracy: 0.75
Precision: 0.4473684210526316
Recall: 0.6538461538461539
AUC: 0.7152209492635025
-----


Training model random_forest on data bert 

Fitting 5 folds for each of 10 candidates, totalling 50 fits
random_forest, bert
F1: 0.4871794871794871
MCC: 0.31567576911753115
Accuracy: 0.6666666666666666
Precision: 0.36538461538461536
Recall: 0.7307692307692307
AUC: 0.6898527004909984
-----


Training model gradient_boosting on data bert 

Fitting 5 folds for each of 10 candidates, totalling 50 fits


# SeqVec Embeddings

In [ ]:
x_chen = pd.read_feather(path.join(DATA_DIR, "chen/embeddings/seqvec/seqvec_chen_embeddings.ftr"))
x_chen_train = x_chen.loc[chen_train.index]
x_chen_test = x_chen.loc[chen_test.index]
x_chen_valid = x_chen.loc[chen_valid.index]
x_chen_train.head()

In [ ]:
try_all(x_chen_train.drop("Ab_ID", axis=1), chen_train["Y"], x_chen_valid.drop("Ab_ID", axis=1), chen_valid["Y"], "seqvec", "2021-11-24", preprocessing=None)